
<h2 align="center">
<FONT "> UNIVERSIDAD AUTÓNOMA DE BAJA CALIFORNIA</FONT><br>
<img src="http://imageninstitucional.uabc.mx/sites/default/files/inline-images/Escudo_0.png" width="200"></h2>


<h3 align="center"><FONT COLOR="black"> Reyes Udasco Rachelle Nerie</FONT><br></h3>
<h3 align="center"><FONT COLOR="black"> Práctica 7</FONT><br></h3>
<h3 align="center"><FONT COLOR="black"> Fecha de entrega: 23 de mayo del 2019</FONT><br></h3>

# Práctica 7: Redes Neuronales

# Introducción

### Redes Neuronales

#### Pasos para la programación del red neuronal

<div align="justify">
    
**1. Inicializar red.** <br>
Crear una red neuronal lista para el entrenamiento.
Defina el número de entradas, el número de neuronas que se deben tener en la capa oculta y el número de salidas.<br>

**2. Propagar hacia adelante.**

Calcule una salida de una red neuronal mediante la propagación de una señal de entrada a través de cada capa hasta que la capa de salida genere sus valores.

Es la técnica que necesitaremos para generar predicciones durante el entrenamiento que deberá ser corregida, y es el método que necesitaremos después de que la red esté capacitada para hacer predicciones sobre nuevos datos.

a. Activación de la neurona.
Calcula la activación de una neurona dada una entrada.
La activación neuronal se calcula como la suma ponderada de las entradas<br>

b. Transferencia de neuronas.
Una vez que se activa una neurona, debemos transferir la activación para ver qué es realmente la salida de la neurona.

Se pueden utilizar diferentes funciones de transferencia. Es tradicional usar la función de activación sigmoide, pero también puede usar la función tanh (tangente hiperbólica) para transferir salidas. <br>

c. Propagación hacia adelante.

Trabajamos a través de cada capa de nuestra red calculando las salidas para cada neurona. Todas las salidas de una capa se convierten en entradas a las neuronas en la siguiente capa.


**3. Propagación hacia atrás.**

El algoritmo de backpropagation se llama así por la forma en que se entrenan los pesos.

a. Derivado de transferencia.
Dado un valor de salida de una neurona, necesitamos calcular su pendiente, por lo que es necesario calcular la derivada de la función. <br>
b. Error Backpropagation.
El primer paso es calcular el error para cada neurona de salida, esto nos dará nuestra señal de error (entrada) para propagarnos hacia atrás a través de la red.


**4. Entrenar red**

La red se entrena usando el descenso de gradiente estocástico.
Esto implica múltiples iteraciones para exponer un conjunto de datos de entrenamiento a la red y para cada fila de datos avanzar propagando las entradas, volviendo a propagar el error y actualizando los pesos de la red.

a. Actualizar pesos.
Una vez que se calculan los errores para cada neurona en la red a través del método anterior de propagación, se pueden usar para actualizar los pesos.<br>
b. Entrenar red

**5. Predicción**
</div>


# Desarrollo

In [230]:
import numpy as np

### Dataset proporcionado

In [200]:
# Entradas del dataset
inputs = np.array([
[4.7,6.0],
[6.1,3.9],
[2.9,4.2],
[7.0,5.5]])

# Salidas del dataset
targets = np.array([
[3.52,4.02],
[5.43,6.23],
[4.95,5.76],
[4.70,4.28]])

# Obtener dimensiones de la entrada
instancias,atributos = inputs.shape
q = instancias
print("Instancias: " + str(instancias),"Atributos: "+ str(atributos))

Instancias: 4 Atributos: 2


### Programación Red Neuronal

In [228]:
 # Funcion de error SSE (Sum of Squares Error)
def sse(target, predicted):
    return np.sum(np.power(np.subtract(target,predicted),2))

# Funcion de transferencia
def funcionTransferencia(activacion):
    return 1/(1+np.exp(-activacion))

# Derivada de la función de transferencia
def dervidadaTransferencia(salida):
    return salida * (1-salida)

class Red_Neuronal():
    def __init__(self, max_iter=50, eta=0.0001, max_error=0.1):
        self.max_iter = max_iter
        self.eta = eta
        self.max_error = max_error

    # Funcion que realiza la activación de la neruona con los pesos y entradas
    def activar(pesos, entradas):
        return np.dot(pesos,entradas)
   
    def propagacionAdelante(self,inputs,pesos):
        a0 = inputs.T
        a0_ = np.concatenate([a0,np.ones((1,q))],axis=0)

        z1 =  activar(pesos[0],a0_)
        a1 =  funcionTransferencia(z1)
        a1_ = np.concatenate([a1,np.ones((1,q))],axis=0)

        z2 =  activar(pesos[1],a1_)
        a2 =  funcionTransferencia(z2)
        a2_ = np.concatenate([a2,np.ones((1,q))],axis=0)

        z3 = activar(pesos[2],a2_)
        a3 = funcionTransferencia(z3)

        return a3,z1,z2,z3

    def propagacionAtras(self,pesos,z1,z2,z3):
        w1_ = pesos[0]
        w2_ = pesos[1]
        w3_ = pesos[2]
        
        deltaK = np.dot(-2*np.exp(1),dervidadaTransferencia(z3))
        deltaJ = np.dot(np.dot(w3_[:,0:3].T,deltaK),dervidadaTransferencia(z2).T)
        deltaI = np.dot(np.dot(w2_[:,0:2].T,deltaJ).T,dervidadaTransferencia(z1))

        #Actualizacion de pesos
        w3New = np.dot(deltaK,a2_)  
        temp = np.concatenate([deltaJ,np.ones((1,3))],axis=0)
        w2New = np.dot(temp.T,a1_.T)
        w1New = np.dot(deltaI,a0.T).T  
        w1_ = w1_ + self.eta *w1New
        w2_ = w2_ + self.eta *w2New
        w3_ = w3_ + self.eta *w3New
        pesosNew = [w1_,w2_,w3_]
        
        return pesosNew


    def fit(self,entradas, targets, pesos):
        
        for i in range(self.max_iter):
            print ("# " + str(i) + "\n")
            a3,z1,z2,z3 = self.propagacionAdelante(inputs,pesos)
            print ("Salida predecida: \n" + str(a3))
            error = sse(targets.T,a3)
            print(pesos)
            pesos = self.propagacionAtras(pesos,z1,z2,z3)

            if error <= self.max_error:
                break
            print("Error = {}".format(error))


In [229]:
RN = Red_Neuronal()
# Valores de los pesos
w1_ = np.random.rand(2,3)
w2_ = np.random.rand(3,3)                      
w3_ = np.random.rand(2,4)
pesos = [w1_,w2_,w3_]

print("Pesos" + str(pesos))
print ("Entrada: \n" + str(inputs))
print ("Salida actual: \n" + str(targets))

RN.fit(inputs, targets, pesos)

Pesos[array([[0.05212946, 0.31314663, 0.25763671],
       [0.7967118 , 0.64604301, 0.59983621]]), array([[0.85495146, 0.35507392, 0.85861574],
       [0.29527093, 0.43311197, 0.5491456 ],
       [0.26277798, 0.33126469, 0.89057832]]), array([[0.66438156, 0.8409969 , 0.40087708, 0.35020847],
       [0.64672067, 0.39255898, 0.79880784, 0.04140106]])]
Entrada: 
[[4.7 6. ]
 [6.1 3.9]
 [2.9 4.2]
 [7.  5.5]]
Salida actual: 
[[3.52 4.02]
 [5.43 6.23]
 [4.95 5.76]
 [4.7  4.28]]
# 0

Salida predecida: 
[[0.87154142 0.87076266 0.87059707 0.87150556]
 [0.82701459 0.82609092 0.82589757 0.82697187]]
[array([[0.05212946, 0.31314663, 0.25763671],
       [0.7967118 , 0.64604301, 0.59983621]]), array([[0.85495146, 0.35507392, 0.85861574],
       [0.29527093, 0.43311197, 0.5491456 ],
       [0.26277798, 0.33126469, 0.89057832]]), array([[0.66438156, 0.8409969 , 0.40087708, 0.35020847],
       [0.64672067, 0.39255898, 0.79880784, 0.04140106]])]
Error = 134.76603515785035
# 1

Salida predecida: 
[[0.87312

C:\Users\7240\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in multiply
  # This is added back by InteractiveShellApp.init_path()


## Conclusión

<div align="justify">
La programación de este ejemplo en específico no fue tan sencillo como se esperaba, no por las funciones en sí que necesitan ser programadas sino por el almacenamiento y la concatencacion de los resultados siendo éstos de diferentes tamaños. Con esto dicho, si fue posible crear las funciones de la propagación hacia adeltante y atrás como se presentó en esta práctica, sin embargo no se obtuvieron resultados precisos por lo que presentaron errores grandes.
</div>

# Referencias

Brownlee, J. (2016) How to Implement the Backpropagation Algorithm From Scratch In Python. Obtenido del Sitio web: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/